<a href="https://colab.research.google.com/github/Moshekwa/MultiClass-Text-Classification/blob/main/Multi_Class_Text_Classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
df = pd.read_excel('/content/WorkingsWinbackSentiments.xlsx')

In [3]:
df.head()

,Template,Membership Number,Suggestions / Compliments / Complaints,Similar Service,Affordability,Unemployed /Lost Job,No Car / Not Driving,Emigrating / Not in Country,Poor Service,Less CallOuts,Separated,Service Not Used,Not Interested,Callback Requested,Not Answered / Voice Mail,Hung Up,Invalid No,Status,Text Analysis
0,T1,524420912,client was not answering her calls i am on the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Closed Unsuccessful,Not Answered / Voice Mail
1,T1,532863208,the client is going through retrenchment and s...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed Successful,Unemployed /Lost Job
2,T1,532910451,the client is mostly out of the country so he...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed Successful,Emigrating / Not in Country
3,T1,532974190,client is currently unemployed at the moment a...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed Successful,Unemployed /Lost Job
4,T1,532999746,client is going through a rough financial situ...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed Successful,Affordability


In [4]:
df = df[['Suggestions / Compliments / Complaints','Text Analysis']]

In [5]:
df.head()

,Suggestions / Compliments / Complaints,Text Analysis
0,client was not answering her calls i am on the...,Not Answered / Voice Mail
1,the client is going through retrenchment and s...,Unemployed /Lost Job
2,the client is mostly out of the country so he...,Emigrating / Not in Country
3,client is currently unemployed at the moment a...,Unemployed /Lost Job
4,client is going through a rough financial situ...,Affordability


# Checking for NULL values and removing them

In [6]:
df['Text Analysis'].unique()

array(['Not Answered / Voice Mail', 'Unemployed /Lost Job',
       'Emigrating / Not in Country', 'Affordability', 'Not Interested',
       'Similar Service', 'No Comments', 'No Car / Not Driving',
       'Hung Up', 'Separated', 'Poor Service', 'Callback Requested',
       'Invalid No', 'Service Not Used', 'Less CallOuts'], dtype=object)

In [7]:
df.isnull().sum()

Suggestions / Compliments / Complaints    0
Text Analysis                             0
dtype: int64

In [8]:
df.fillna("unknown",axis=1)

,Suggestions / Compliments / Complaints,Text Analysis
0,client was not answering her calls i am on the...,Not Answered / Voice Mail
1,the client is going through retrenchment and s...,Unemployed /Lost Job
2,the client is mostly out of the country so he...,Emigrating / Not in Country
3,client is currently unemployed at the moment a...,Unemployed /Lost Job
4,client is going through a rough financial situ...,Affordability
...,...,...
338,the member said he cancelled because he has th...,Similar Service
339,Client says that we must call him after 3 mont...,Unemployed /Lost Job
340,there are no contact details here so i didnt call,Invalid No
341,client says that she cant afford she says she ...,Affordability


In [9]:
#df.isnull().sum()
#df.cuisine.unique()

In [10]:
#df.dropna(how ='all')

In [11]:
#sumNan = df.isna()
#sumNan.sum()

# Dropping Duplicates

In [12]:
df.shape

(343, 2)

In [13]:
df.drop_duplicates(inplace=True)

In [14]:
df.shape


(322, 2)

# Preprocessing the text

In [15]:
print(df['Suggestions / Compliments / Complaints'].apply(lambda x: len(x.split(' '))).sum())

6066


In [16]:
special_character_remover = re.compile('[/(){}\[\]\|@,;]')
extra_symbol_remover = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

## Stemming and Lemmatization

In [17]:
def clean_text(text):
    text = text.lower()
    text = special_character_remover.sub(' ', text)
    text = extra_symbol_remover.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text
    
df['Suggestions / Compliments / Complaints'] = df['Suggestions / Compliments / Complaints'].apply(clean_text)

In [18]:
print(df['Suggestions / Compliments / Complaints'].apply(lambda x: len(x.split(' '))).sum())

2911


# Train Test Split

In [19]:
from sklearn.model_selection import train_test_split
X = df['Suggestions / Compliments / Complaints']
y = df['Text Analysis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [20]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((225,), (97,), (225,), (97,))

# Logictic Regression

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

lr = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', LogisticRegression()),])

lr.fit(X_train,y_train)
y_pred1 = lr.predict(X_test)

print(f"Accuracy is : {accuracy_score(y_pred1,y_test)}")

Accuracy is : 0.6391752577319587


In [37]:
from google.colab import drive
drive.mount('/content/drive')

# list the directories in the home directory
import os
os.listdir('.')
g_path = '/content/data/'
drive = '/content/drive/MyDrive/'  
# move the sub-directory (data) into google-drive
mv g_path drive

SyntaxError: ignored

In [41]:
from sklearn.externals import joblib
# Save the model as a pickle in a file
joblib.dump(lr, 'filename.pkl')
from google.colab import files
files.download("filename.pkl")
# Load the model from the file
#knn_from_joblib = joblib.load('filename.pkl')
 
# Use the loaded model to make predictions
#knn_from_joblib.predict(X_test)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Logistic Regression Metrics

In [22]:
from sklearn import metrics 

In [23]:
print (f"Accuracy: {round(metrics.accuracy_score(y_test,y_pred1)*100,2)}%")
print (f"Precision Score: {round(metrics.precision_score(y_test,y_pred1,pos_label='positive', average='micro')*100,2)}%")
print (f"Recall: { round(metrics.recall_score(y_test,y_pred1,pos_label='positive',average='micro')*100,2)} %" )

print(metrics.classification_report(y_test, y_pred1))

Accuracy: 63.92%
Precision Score: 63.92%
Recall: 63.92 %
                           precision    recall  f1-score   support

            Affordability       0.62      0.56      0.59         9
       Callback Requested       0.00      0.00      0.00         2
                  Hung Up       0.50      0.40      0.44         5
               Invalid No       1.00      0.60      0.75         5
     No Car / Not Driving       1.00      0.60      0.75         5
              No Comments       1.00      0.27      0.43        11
Not Answered / Voice Mail       0.67      0.93      0.78        15
           Not Interested       0.71      0.80      0.75        15
             Poor Service       0.00      0.00      0.00         3
                Separated       0.00      0.00      0.00         1
          Similar Service       0.47      1.00      0.64        16
     Unemployed /Lost Job       1.00      0.40      0.57        10

                 accuracy                           0.64        97
   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Confusion Matrix

In [24]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, y_pred1)
print(matrix)

[[ 5  0  0  0  0  0  0  0  0  0  4  0]
 [ 0  0  1  0  0  0  1  0  0  0  0  0]
 [ 0  0  2  0  0  0  3  0  0  0  0  0]
 [ 0  0  0  3  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  3  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  3  0  3  0  0  5  0]
 [ 0  0  0  0  0  0 14  0  0  0  1  0]
 [ 0  0  1  0  0  0  0 12  0  0  2  0]
 [ 0  0  0  0  0  0  0  2  0  0  1  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 16  0]
 [ 3  0  0  0  0  0  0  0  0  0  3  4]]


In [25]:
# Defining Accuracy as the sum of all the values in the diagonal over the total number of observations
correctPredictions = sum(matrix[i][i] for i in range(len(matrix)))
print(correctPredictions)
print(f'Number of observations: {len(y_test)}')
print("Accuracy: {:.2}".format(correctPredictions / len(y_test)))

62
Number of observations: 97
Accuracy: 0.64


# Testing With Logistic Regression

In [26]:
x = ['client is not answering the calls no matter how many times i tried to call']
y_ = lr.predict(x)
print(y_)

['Not Answered / Voice Mail']


# Naive Bayes Classifier

In [27]:
from sklearn.naive_bayes import MultinomialNB

naivebayes = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])
naivebayes.fit(X_train, y_train)

y_pred = naivebayes.predict(X_test)

print(f'accuracy {accuracy_score(y_pred,y_test)}')

accuracy 0.4948453608247423


# Applying Xgboost Classifier

In [28]:
from xgboost import XGBClassifier

xgboost = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', XGBClassifier()),
              ])
xgboost.fit(X_train, y_train)

y_pred = xgboost.predict(X_test)

print(f'accuracy {accuracy_score(y_pred,y_test)}')

accuracy 0.6494845360824743


In [29]:
x = ['Unfortunately client does not have a car at the moment']
y_ = xgboost.predict(x)
print(y_)

['No Car / Not Driving']


# Save model
import pickle
with open('model.pkl','wb') as f:
    pickle.dump(text_clf,f

# load model    
# load
with open('model.pkl', 'rb') as f:
    clf2 = pickle.load(f)

docs_new = ['Ten Months After George Floyd’s Death, Minneapolis Residents Are at War Over Policing']
predicted = clf2.predict(docs_new)

v[predicted[0]]